# Librerias

In [1]:
import pandas
import matplotlib.pyplot as plt

# Funciones de lectura de Información:

La función "lecturaestacion", lee los archivos de la estación a analizar y obtiene los datos entre
Enero 2018 y Diciembre 2019.

In [2]:
def lecturaestacion(numero,elemento):
    fechas = ('201802','201803','201804','201805','201806','201807','201808','201809','201810','201811','201812',
        '201901','201902','201903','201904','201905','201906','201907','201908','201909','201910','201911','201912')
    Datos = pandas.read_csv('/home/matias/Escritorio/2Meteorologicas/'+numero+'/'+numero+'_201801_'+elemento+'.csv',sep=';')
    for fec in fechas:
        direccion = '/home/matias/Escritorio/2Meteorologicas/'+numero+'/'+numero+'_'+fec+'_'+elemento+'.csv'
        Datos2 = pandas.read_csv(direccion,sep=';')
        Datos = pandas.concat([Datos,Datos2])
    Largo_BD=len(Datos)
    columnas=list(Datos.columns.values)
    for col in columnas:
        if Datos[col].isna().sum()==Largo_BD:
            Datos=Datos.drop(columns=[col])
    if Datos['idEquipo'].sum()==0:
        Datos=Datos.drop(columns=['idEquipo'])
    return Datos

La función "leerbases" implementa la función "lecturaestacion" para cada una de las variables.

In [3]:
def leerbases(numero):
    return {'Agua':lecturaestacion(numero,'AguaCaida'),'PrHu':lecturaestacion(numero,'PresionHumedad'),
           'Radi':lecturaestacion(numero,'RadiacionGlobal'),'Temp':lecturaestacion(numero,'Temperatura'),
           'Wind':lecturaestacion(numero,'Viento')}

La función "Armarbase" une la información de cada uno de los archivos obtenidos por función "leerbases" en un dataframe.

In [4]:
def Armarbase(numero):
    Bases=leerbases(numero)
    Agua=Bases['Agua'].set_index('momento').loc[:,['rrInst','rr6Horas','rr24Horas']]
    PrHu=Bases['PrHu'].set_index('momento').loc[:,['hr','p0','qfe1','qff','qnh']]
    Radi=Bases['Radi'].set_index('momento').loc[:,['radiacionGlobalInst','radiacionGlobalAcumulada24Horas']]
    Wind=Bases['Wind'].set_index('momento').loc[:,['ddInst','ffInst','dd02Minutos','ff02Minutos','dd10Minutos','ff10Minutos']]
    Temp=Bases['Temp'].set_index('momento').loc[:,['ts','td','tMin12Horas','tMax12Horas','horaTMin12Horas','horaTMax12Horas']]
    Base=Agua.join(PrHu,how='outer').join(Radi,how='outer').join(Wind,how='outer').join(Temp,how='outer')
    Base2=Instantes(Base)
    return Base2

La función "Instantes" es utilizada por función "Armarbase" para desglosar las variables temporales como años, meses, días, horas y minutos, y, calcular Hora_Min_t y Hora_Max_t.

In [5]:
def Instantes(Bases):
    Aux=Bases.reset_index()
    Aux=Aux[Aux.isna().sum(1)==0]
    Aux['Anno'] = pandas.to_datetime(Aux['momento']).map(lambda x: x.year).astype('uint16')
    Aux['Mes'] = pandas.to_datetime(Aux['momento']).map(lambda x: x.month).astype('uint8')
    Aux['Dia'] = pandas.to_datetime(Aux['momento']).map(lambda x: x.day).astype('uint8')
    Aux['Hora'] = pandas.to_datetime(Aux['momento']).map(lambda x: x.hour).astype('uint8')
    Aux['Minuto'] = pandas.to_datetime(Aux['momento']).map(lambda x: x.minute).astype('uint8')
    Aux['momento']=pandas.to_datetime(Aux['momento'])
    Aux['Hora_Min_t'] = pandas.to_datetime(Aux['horaTMin12Horas']).map(lambda x: x.hour).astype('uint8')
#    Aux['Minuto_Min_t'] = pandas.to_datetime(Aux['horaTMin12Horas']).map(lambda x: x.minute).astype('uint8')
    Aux['Hora_Max_t'] = pandas.to_datetime(Aux['horaTMax12Horas']).map(lambda x: x.hour).astype('uint8')
#    Aux['Minuto_Max_t'] = pandas.to_datetime(Aux['horaTMax12Horas']).map(lambda x: x.minute).astype('uint8')
    Aux=Aux.drop(columns=['horaTMin12Horas','horaTMax12Horas'])
    return Aux

# Funciones Para mostrar la base de datos por estación:

La función "DibujaCorr" grafica las correlaciones entre las variables.

In [6]:
def Dibujacorr(BD):
    plt.matshow(abs(BD.corr()))
    plt.xticks(range(len(BD.columns)), BD.columns)
    plt.yticks(range(len(BD.columns)), BD.columns)
    plt.colorbar()
    plt.show()
    print(BD.corr()['ts'].round(3))
    print('NANs')
    display(BD.isnull().sum())
    display(BD)

La función "dibujarbd" muestra las variables contra el tiempo.

In [7]:
def dibujarbd(BD):
    x=BD['momento']
    for k in ('rrInst','rr6Horas','rr24Horas'):
        plt.scatter(x,BD[k],s=0.4,label=k)
    plt.legend(loc='upper right', shadow=True, fontsize='x-large')
    plt.xticks(rotation=60)
    plt.show()
    for k in ('hr','p0','qfe1','qff','qnh'):
        plt.scatter(x,BD[k],s=0.4,label=k)
    plt.legend(loc='upper right', shadow=True, fontsize='x-large')
    plt.xticks(rotation=60)
    plt.show()
    for k in ('radiacionGlobalInst','radiacionGlobalAcumulada24Horas'):
        plt.scatter(x,BD[k],s=0.4,label=k)
    plt.legend(loc='upper right', shadow=True, fontsize='x-large')
    plt.xticks(rotation=60)
    plt.show()
    for k in ('ddInst','ffInst','dd02Minutos','ff02Minutos','dd10Minutos','ff10Minutos'):
        plt.scatter(x,BD[k],s=0.4,label=k)
    plt.legend(loc='upper right', shadow=True, fontsize='x-large')
    plt.xticks(rotation=60)
    plt.show()
    for k in ('ts','td','tMin12Horas','tMax12Horas'):
        plt.scatter(x,BD[k],s=0.4,label=k)
    plt.legend(loc='upper right', shadow=True, fontsize='x-large')
    plt.xticks(rotation=60)
    plt.show()
    Dibujacorr(BD)

La función "graficar_densidades" muestra gráficamente las densidades de las variables.

In [8]:
def graficar_densidades(BD,bins=24):
    if ('momento' in BD.columns):
        BD2=BD.drop(columns=['momento'])
    else:
        BD2=BD
    largo=len(BD2)
    display(BD2.describe())
    for k in BD2.columns:
        print(k)
        aux=pandas.DataFrame()
        aux.loc['Máximo','Valor']=round(BD2[k].max(),4)
        aux.loc['Mínimo','Valor']=round(BD2[k].min(),4)
        aux.loc['Media','Valor']=round(BD2[k].mean(),4)
        aux.loc['Mediana','Valor']=round(BD2[k].median(),4)
        aux.loc['Desviación Estándar','Valor']=round(BD2[k].std(),4)
        aux.loc['Asimetría','Valor']=round(sum(aux.loc['Mediana','Valor']-BD2[k])/aux.loc['Desviación Estándar','Valor']/len(BD2[k]),4)
        print(aux)
        plt.hist(BD2[k],label=k,bins=bins)
        plt.legend(loc='upper right', shadow=True, fontsize='x-large')
        plt.xticks(rotation=60)
        plt.show()

# A Continuación se muestran cada una de las bases de datos utilizando las funciones recién construidas. 

In [9]:
BD=Armarbase('340115')#Pichilemu
BD.to_csv('Pichilemu.csv',sep='|')

In [10]:
BD=Armarbase('340066')#Nilahue
BD.to_csv('Nilahue.csv',sep='|')

In [11]:
BD=Armarbase('350028')#Panguilemo
BD.to_csv('Panguilemo.csv',sep='|')

In [12]:
BD=Armarbase('360047')#Retiro
BD.to_csv('Retiro.csv',sep='|')

In [13]:
BD=Armarbase('360046')#Chillan
BD.to_csv('Chillan.csv',sep='|')

In [14]:
BD=Armarbase('380013')#Maquehue_Temuco
BD.to_csv('Maquehue_Temuco.csv',sep='|')

In [15]:
BD=Armarbase('380033')#Maquehue_ufro
BD.to_csv('Maquehue_ufro.csv',sep='|')

In [16]:
BD=Armarbase('390029')#Panguipulli
BD.to_csv('Panguipulli.csv',sep='|')